Load packages

In [1]:
import os
os.chdir('../../')

In [2]:
cd = os.getcwd()
functions_path = os.path.join(cd, '02_code/functions')

In [6]:
import scanpy as sc
import scanpy.external as sce
import numpy as np
import pandas as pd
import anndata as ad
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from anndata import AnnData
import skmisc
import regex as re

import anndata2ri
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

# from scipy.sparse import csr_matrix
import sys
sys.path.append(functions_path)
import functions_dat_processing as dp
import functions_plotting as dplt
import functions_for_CAR_annotation as ca

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


/tmp/ipykernel_2506308/1078855118.py:19: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [7]:
#if needed, reload functions
import importlib
importlib.reload(dp)
importlib.reload(dplt)
importlib.reload(ca)

<module 'functions_for_CAR_annotation' from '/home/s377963/car_t_sc/02_code/functions/functions_for_CAR_annotation.py'>

Import data

In [4]:
# adata_raw = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_calagry_exact_TIL_only.h5ad') #prior to any sort of processing, shows maximal possible yield with respect to CAR-T annotation
# adata_dLN = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_calagry_exact_TIL_only.h5ad') #after dLN filtering
adata_annotated = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_1xintegrated_scaled_pureTCs_annotated_calagry_exact.h5ad') #after annotating TCs and TC subtypes